# Feature Selection

In [1]:
from feature_search.SearchNodes import SearchNodeAdding
import configuration
from feature_search.SplitConfig import SplitConfig
from feature_search.ModelConfig import ModelConfigCNN
from modeling.Trainer import Trainer

In [2]:
from data_loading import load_data

data = load_data()

In [3]:


window_size = 104

model_config = ModelConfigCNN(window_size=window_size,
                              num_features= 1,
                              output_size=1,
                              num_conv = 3,
                              kernel_size = 7,
                              channels=2,
                              padding=True,
                              num_hidden_layers=2,
                              hidden_size=50,
                              dropout=0.2
                              )

split_config = SplitConfig(window_size=window_size,
                           prediction_length=1,
                           look_ahead=1,
                           val_split=configuration.validation_split,
                           test_split=configuration.test_split,
                           is_indexed=True,
                           scale_target=True,
                           fixed_feature_size=None,
                           batch_size=250,
                           )


model_trainer = Trainer(epochs = 100, learning_rate=1e-4)


In [4]:
features = list(set(data.columns) - {configuration.target_column, configuration.date_column})
features

['NYF',
 'Y10',
 'UN',
 'MOV ',
 'PE',
 'YSS',
 'Rho',
 'Y02',
 '_DXY',
 'RV',
 'IR',
 'CPI',
 'M2',
 '_LCP',
 'GDP',
 'DY',
 'STP',
 'ED',
 '_VA',
 'EMP',
 'CF',
 '_GR',
 '_OIL',
 'RR',
 'MG',
 '_AU',
 'CAPE',
 '_TY',
 'DIL']

In [5]:
node = SearchNodeAdding(data,
                        features=features,
                        selection=['_VA'],
                        model_trainer=model_trainer,
                        target_column=configuration.target_column,
                        date_column=configuration.date_column,
                        model_config=model_config,
                        split_config=split_config,
                        num_iterations=8,
                        parent=None
                        )

In [6]:
import datetime
start = datetime.datetime.now()

print(node.get_heuristic())

end = datetime.datetime.now()

print(end - start)

0.6404895676542024
0.46017963516301125
0.8805416841405772
0.4403141167024719
2.0187369480689576
0.21705008758632194
0.5281632523637868
0.18691040875899065
0.6715482125547899
0:00:46.905481


In [7]:
children = node.get_children()

In [8]:
for i in children: 
    print(f"{i.selection}: {round(i.get_heuristic(),3)}")

0.327650575169834
0.4029623823393561
0.33222564367147595


KeyboardInterrupt: 